In [1]:
import sys
sys.path.append('/home/fyjj/Public-DSC291/Public-DSC291/notebooks/Section2-PCA/PCA/lib/')

import numpy as np
from time import time
from lib.numpy_pack import packArray,unpackArray
from lib.spark_PCA import computeCov
from lib.computeStatistics import *
from lib.YearPlotter import YearPlotter
import pandas as pd

import findspark
findspark.init()
import os
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
!aws s3 ls s3://philipp-ghcnd

                           PRE GHCND.parquet/
                           PRE GHCND_WT08_1960_2020_US.parquet/
                           PRE GHCNDall.parquet/
                           PRE ghcnblog/
                           PRE stations_raw/
2020-05-16 05:26:43          0 GHCND.parquet_$folder$
2020-05-17 22:44:28          0 GHCND_WT08_1960_2020_US.parquet_$folder$
2020-05-16 07:03:22          0 GHCNDall.parquet_$folder$


In [6]:
# SparkConf().set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0-alpha3")
sc = SparkContext(master="local[4]")
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet("/home/fyjj/weather_data")

In [9]:
print('number of rows',df.cache().count())
df.show(5)

number of rows 32295
+-----------+----+--------------------+
|    Station|year|              Values|
+-----------+----+--------------------+
|USW00014735|1968|[0, 0, 1, 1, 0, 0...|
|USW00014735|1969|[0, 0, 0, 0, 0, 1...|
|USW00014735|1970|[0, 0, 0, 1, 1, 1...|
|USW00014735|1971|[0, 0, 0, 1, 0, 0...|
|USW00014735|1972|[0, 0, 0, 1, 0, 0...|
+-----------+----+--------------------+
only showing top 5 rows



In [35]:
df.describe().show()

+-------+-----------+------------------+--------------------+
|summary|    Station|              year|              Values|
+-------+-----------+------------------+--------------------+
|  count|      32295|             32295|               32295|
|   mean|       null|1993.4481498684006|                null|
| stddev|       null|19.562831340768035|                null|
|    min|USC00012971|              1960|[0, 0, 0, 0, 0, 0...|
|    max|USW00094994|              2020|[1, 1, 1, 1, 1, 1...|
+-------+-----------+------------------+--------------------+



In [63]:
uni_station = df.groupBy('Station').agg({'Station':'count','year':'min'}).toPandas()

In [64]:
uni_station.head()

,Station,count(Station),min(year)
0,USC00033466,1,1963
1,USC00108818,2,1962
2,USC00124527,1,1963
3,USC00172878,2,1962
4,USC00193624,1,1962


In [67]:
uni_station.max()

Station           USW00094994
count(Station)             61
min(year)                2020
dtype: object

In [76]:
uni_station['min(year)']=uni_station['min(year)'].astype(int)
uni_station[uni_station['count(Station)']+uni_station['min(year)']>2019].count()

Station           540
count(Station)    540
min(year)         540
dtype: int64

In [78]:
station_eff = uni_station[uni_station['count(Station)']+uni_station['min(year)']>2019]
valid_years = station_eff['count(Station)']
work_len_cut= [5,10,15,20,25,30,35,40,45,50]
cats = pd.cut(valid_years,work_len_cut)
pd.value_counts(cats)

(10, 15]    301
(15, 20]     19
(20, 25]     15
(45, 50]      3
(35, 40]      3
(25, 30]      3
(30, 35]      1
(40, 45]      0
(5, 10]       0
Name: count(Station), dtype: int64